In [74]:
from math import exp, sqrt
class Crr:
    def __init__(self, init_price, t, delta, r, sigma=None, u=None, d=None):
        """Initialize the Cox-Ross-Rubinstein model"""
        self.init_price=init_price
        self.sigma=sigma
        self.t=t  # terminal time
        self.delta=delta # delta t, e.g.: 1/12 if the price changes each month
        self.u=exp(self.sigma*sqrt(self.delta)) if sigma else u
        self.d=exp(-self.sigma*sqrt(self.delta)) if sigma else d       
        self.r=r
        self.q=(exp(self.r*self.delta)-self.d)/(self.u-self.d)
        self.tree=None
        
    def create(self, current_depth=0, price=None):
        """This method creates a dictionary which represents the binomial tree"""
        if price==None:
            price=self.init_price
        steps=self.t/self.delta
        if current_depth >= steps:
            return {"price": price, "isleaf":True}
        up_tree = self.create(current_depth + 1, price*self.u)        
        down_tree = self.create(current_depth + 1, price*self.d)
        return {'price': price, 'up': up_tree, 'down': down_tree, 'isleaf':False}
    
    def add_option(self, tree, strike):
        """Adds terminal payoff of the call option to the leafs of the tree"""
        if tree['isleaf']:
            tree["option"]=max(tree['price']-strike,0 )
        else:
            self.add_option(tree['up'], strike)
            self.add_option(tree['down'], strike)
            #tree["option"]=None
            
    def add_put_option(self, tree, strike):
        """Adds terminal payoff of the put option to the leafs of the tree"""
        if tree['isleaf']:
            tree["option"]=max(strike-tree['price'],0 )
        else:
            self.add_put_option(tree['up'], strike)
            self.add_put_option(tree['down'], strike)
            #tree["option"]=None
            
    def option_price(self, tree):
        """gives the price of the European option"""
        if tree['isleaf']:
            return tree["option"]
        return exp(-self.r*self.delta)*(self.q*self.option_price(tree['up'])+ \
                              (1-self.q)*self.option_price(tree['down']))
    
    def american_call_option_price(self, tree, strike):
        """gives the price of the American option"""
        if tree['isleaf']:
            return tree["option"]
        return max(tree["price"]-strike, exp(-self.r*self.delta)* \
                   (self.q*self.american_put_option_price(tree['up'], strike)+ \
                        (1-self.q)*self.american_put_option_price(tree['down'], strike)))
    def american_put_option_price(self, tree, strike):
        """gives the price of the American option"""
        if tree['isleaf']:
            return tree["option"]
        return max(strike-tree["price"], exp(-self.r*self.delta)* \
                   (self.q*self.american_put_option_price(tree['up'], strike)+ \
                        (1-self.q)*self.american_put_option_price(tree['down'], strike)))

    def price(self, strike, put=False, american=False):
        """Cretaes the tree, adds terminal payoff to the tree and gives prices of any option"""
        if self.tree==None:
            self.tree=self.create()
        tree=dict(self.tree)
        if put==False:
            self.add_option(tree, strike)
            if american==False:
                price=self.option_price(tree)
            else:
                price=self.american_put_option_price(tree, strike)
        else:
            self.add_put_option(tree, strike)
            if american==False:
                price=self.option_price(tree)
            else:
                price=self.american_put_option_price(tree, strike)            
        return price



In [75]:
crr=Crr(50, 2, 1, 0.05, 0.3)
#print "call, European: ", crr.price(52)
print "put, European: ", crr.price(52, True)
#print "call, American: ", crr.price(52, False, True)
print "put, American: ", crr.price(52, True, True)  #Hall: 7th edition p.: 250-251

put, European:  6.24570844521
put, American:  7.4284019027


In [76]:
tree=crr.create()

print tree

{'down': {'down': {'price': 27.440581804701324, 'isleaf': True}, 'price': 37.040911034085894, 'up': {'price': 50.00000000000001, 'isleaf': True}, 'isleaf': False}, 'price': 50, 'up': {'down': {'price': 50.0, 'isleaf': True}, 'price': 67.49294037880016, 'up': {'price': 91.10594001952546, 'isleaf': True}, 'isleaf': False}, 'isleaf': False}


In [77]:

print tree["price"], tree["up"]["price"], tree["up"]["up"]["price"]

50 67.4929403788 91.1059400195


In [78]:
#a simple model, without giving sigma (sigma=None)
#Hall: 7th edition, p 246:
crr246=Crr(50, 2, 1, 0.05, None, 1.2,0.8)
print "put, European: ", crr246.price(52, True)

put, European:  4.1926542806


In [79]:
#Hall: 7th edition, p 247:
crr247=Crr(50, 2, 1, 0.05, None, 1.2,0.8)
print "put, American: ", crr247.price(52, True, True)

put, American:  5.0896324742


In [80]:
#Hall: 7th edition, p 251 five steps:
crr251=Crr(50, 2, 0.4, 0.05, 0.3) # numer of steps=2/0.4
print "put, American: ", crr251.price(52, True, True)

put, American:  7.67088873475


In [81]:
#p 252 five steps:
crr252=Crr(810, 0.5, 0.25, 0.03, 0.2) # numer of steps=0.5/0.25
print crr252.q 
print crr252.u
print "call, European: ", crr252.price(800, False, False)  # 53.93 not match book's result 53.39, typo?

0.512599127895
1.10517091808
call, European:  53.9313421959


In [82]:
#Hall: 7th edition, p 253 options on currencies:
crr253=Crr(0.61, 0.25, 1.0/12, -0.02, 0.12) # numer of steps=0.25/1/12
print crr253.q  #doesn't match
print crr253.u
print "put, American: ", crr253.price(0.6, True, True) #0.0119 doesn't match 0.019 typo?
print crr253.tree["price"], crr253.tree["up"]["price"], crr253.tree["up"]["up"]["price"]

0.467309192991
1.03524800477
put, American:  0.0119332055349
0.61 0.631501282911 0.653760443145


In [83]:
#Hall: 7th edition, p 255 american put option  on futures:
crr255=Crr(31, 0.75, 0.25, 0.05, 0.3) # numer of steps=0.5/0.25
# beacuse in a risk neutral world, the expected growth rate of futures price should be zero
crr255.q=(1-crr255.d)/(crr255.u-crr255.d)
print crr255.q  #doesn't match
print crr255.u
print "put, American: ", crr255.price(30, True, True) 
print crr255.tree["price"], crr255.tree["up"]["price"], crr255.tree["up"]["up"]["price"], crr255.tree["up"]["up"]["up"]["price"]

0.462570154656
1.16183424273
put, American:  2.83563515711
31 36.0168615246 41.8456230349 48.6176777502
